In [2]:
import math
import re
import heapq
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname = "C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family = font_name)
import seaborn as sns

import warnings
warnings.filterwarnings(action = "ignore", category = FutureWarning)

from scipy.optimize import curve_fit

## 노드맵 그리기

+ 서울시 각 구에 각각의 노드를 생성해 이어줌으로서 그래프를 생성 - 계 : 118개의 노드
    - 강남구 : 8개
        + 한강노드, 공항
    - 강동구 : 4개
    - 강북구 : 4개
    - 강서구 : 5개
        + 한강노드, 공항
    - 관악구 : 5개
    - 광진구 : 3개
    - 구로구 : 5개
    - 금천구 : 3개
    - 노원구 : 2개
    - 도봉구 : 2개
    - 동대문구 : 5개
    - 동작구 : 4개
        + 한강노드
    - 마포구 : 5개
        + 한강노드, 여의도
    - 서대문구 : 4개
    - 서초구 : 6개
        + 한강노드
    - 성동구 : 5개
        + 한강노드
    - 성북구 : 5개
    - 송파구 : 6개
        + 한강노드
    - 양천구 : 3개
    - 영등포구 : 6개
        + 한강노드, 여의도
    - 용산구 : 6개
        + 한강노드, KTX, 여의도
    - 은평구 : 5개
    - 종로구 : 6개
    - 중구 : 5개
        + KTX
    - 중랑구 : 3개
    - 여의도
    - 한강
    - 서울시 외각

In [127]:
graph_seoul2 = {'강남구1' : set(['강남구2', '강남구3', '강남구4', '강남구5', '강남구6', '강남구7', '강남구8', '용산구1', '성동구1']),
               '강남구2' : set(['강남구1', '강남구3', '강남구4', '강남구5', '강남구6', '강남구7', '강남구8', '서울시 외곽']),
               '강남구3' : set(['강남구1', '강남구2', '강남구4', '강남구5', '강남구6', '강남구7', '강남구8', '서초구6']),
               '강남구4' : set(['강남구1', '강남구2', '강남구3', '강남구5', '강남구6', '강남구7', '강남구8', '서초구5']),
               '강남구5' : set(['강남구1', '강남구2', '강남구3', '강남구4', '강남구6', '강남구7', '강남구8', '송파구6']),
               '강남구6' : set(['강남구1', '강남구2', '강남구3', '강남구4', '강남구5', '강남구7', '강남구8', '송파구5']),
               '강남구7' : set(['강남구1', '강남구2', '강남구3', '강남구4', '강남구5', '강남구6', '강남구8', '송파구4']),
               '강남구8' : set(['강남구1', '강남구2', '강남구3', '강남구4', '강남구5', '강남구6', '강남구7']),
               '강동구1' : set(['강동구2', '강동구3', '강동구4', '서울시 외곽']),
               '강동구2' : set(['강동구1', '강동구3', '강동구4', '서울시 외곽']),
               '강동구3' : set(['강동구1', '강동구2', '강동구4', '송파구3']),
               '강동구4' : set(['강동구1', '강동구2', '강동구3', '송파구2']),
               '강북구1' : set(['강북구2', '강북구3', '강북구4', '서울시 외곽']),
               '강북구2' : set(['강북구1', '강북구3', '강북구4', '도봉구2']),
               '강북구3' : set(['강북구1', '강북구2', '강북구4', '성북구2']),
               '강북구4' : set(['강북구1', '강북구2', '강북구3', '성북구1']),
               '강서구1' : set(['강서구2', '강서구3', '강서구4', '강서구5', '마포구1']),
               '강서구2' : set(['강서구1', '강서구3', '강서구4', '강서구5', '서울시 외곽']),
               '강서구3' : set(['강서구1', '강서구2', '강서구4', '강서구5', '서울시 외곽']),
               '강서구4' : set(['강서구1', '강서구2', '강서구3', '강서구5', '양천구3']),
               '강서구5' : set(['강서구1', '강서구2', '강서구3', '강서구4']),
               '관악구1' : set(['관악구2', '관악구3', '관악구4', '관악구5', '서울시 외곽']),
               '관악구2' : set(['관악구1', '관악구3', '관악구4', '관악구5', '금천구3']),
               '관악구3' : set(['관악구1', '관악구2', '관악구4', '관악구5', '구로구4']),
               '관악구4' : set(['관악구1', '관악구2', '관악구3', '관악구5', '동작구3']),
               '관악구5' : set(['관악구1', '관악구2', '관악구3', '관악구4', '서초구3']),
               '광진구1' : set(['광진구2', '광진구3', '송파구1']),
               '광진구2' : set(['광진구1', '광진구3', '성동구5']),
               '광진구3' : set(['광진구1', '광진구2', '중랑구2', '동대문구3']),
               '구로구1' : set(['구로구2', '구로구3', '구로구4', '구로구5', '서울시 외곽']),
               '구로구2' : set(['구로구1', '구로구3', '구로구4', '구로구5', '양천구2']),
               '구로구3' : set(['구로구1', '구로구2', '구로구4', '구로구5', '영등포구5']),
               '구로구4' : set(['구로구1', '구로구2', '구로구3', '구로구5', '관악구3']),
               '구로구5' : set(['구로구1', '구로구2', '구로구3', '구로구4', '금천구2']),
               '금천구1' : set(['금천구2', '금천구3', '서울시 외곽']),
               '금천구2' : set(['금천구1', '금천구3', '구로구5']),
               '금천구3' : set(['금천구1', '금천구2', '영등포구4', '관악구2']),
               '노원구1' : set(['노원구2', '중랑구3', '성북구3']),
               '노원구2' : set(['노원구1', '도봉구1']),
               '도봉구1' : set(['도봉구2', '노원구2']),
               '도봉구2' : set(['도봉구1', '강북구2']),
               '동대문구1' : set(['동대문구2', '동대문구3', '동대문구4', '동대문구5', '성북구3']),
               '동대문구2' : set(['동대문구1', '동대문구3', '동대문구4', '동대문구5', '중랑구2']),
               '동대문구3' : set(['동대문구1', '동대문구2', '동대문구4', '동대문구5', '광진구3']),
               '동대문구4' : set(['동대문구1', '동대문구2', '동대문구3', '동대문구5', '성동구4', '종로구3', '중구2']),
               '동대문구5' : set(['동대문구1', '동대문구2', '동대문구3', '동대문구4', '성북구4']),
               '동작구1' : set(['동작구2', '동작구3', '동작구4', '용산구1']),
               '동작구2' : set(['동작구1', '동작구3', '동작구4', '서초구4']),
               '동작구3' : set(['동작구1', '동작구2', '동작구4', '관악구4']),
               '동작구4' : set(['동작구1', '동작구2', '동작구3', '영등포구3']),
               '마포구1' : set(['마포구2', '마포구3', '마포구4', '마포구5', '강서구1', '영등포구1']),
               '마포구2' : set(['마포구1', '마포구3', '마포구4', '마포구5', '여의도']),
               '마포구3' : set(['마포구1', '마포구2', '마포구4', '마포구5', '은평구5']),
               '마포구4' : set(['마포구1', '마포구2', '마포구3', '마포구5', '서대문구3']),
               '마포구5' : set(['마포구1', '마포구2', '마포구3', '마포구4', '중구4', '용산구3']),
               '서대문구1' : set(['서대문구2', '서대문구3', '서대문구4', '종로구5']),
               '서대문구2' : set(['서대문구1', '서대문구3', '서대문구4', '중구4']),
               '서대문구3' : set(['서대문구1', '서대문구2', '서대문구4', '마포구4']),
               '서대문구4' : set(['서대문구1', '서대문구2', '서대문구3', '은평구4']),
               '서초구1' : set(['서초구2', '서초구3', '서초구4', '서초구5', '서초구6', '용산구1']),
               '서초구2' : set(['서초구1', '서초구3', '서초구4', '서초구5', '서초구6', '서울시 외곽']),
               '서초구3' : set(['서초구1', '서초구2', '서초구4', '서초구5', '서초구6', '관악구5']),
               '서초구4' : set(['서초구1', '서초구2', '서초구3', '서초구5', '서초구6', '동작구2']),
               '서초구5' : set(['서초구1', '서초구2', '서초구3', '서초구4', '서초구6', '강남구4']),
               '서초구6' : set(['서초구1', '서초구2', '서초구3', '서초구4', '서초구5', '강남구3']),
               '성동구1' : set(['성동구2', '성동구3', '성동구4', '성동구5', '강남구1']),
               '성동구2' : set(['성동구1', '성동구3', '성동구4', '성동구5', '용산구5']),
               '성동구3' : set(['성동구1', '성동구2', '성동구4', '성동구5', '중구2']),
               '성동구4' : set(['성동구1', '성동구2', '성동구3', '성동구5', '동대문구4']),
               '성동구5' : set(['성동구1', '성동구2', '성동구3', '성동구4', '광진구2']),
               '성북구1' : set(['성북구2', '성북구3', '성북구4', '성북구5', '강북구4']),
               '성북구2' : set(['성북구1', '성북구3', '성북구4', '성북구5', '강북구3']),
               '성북구3' : set(['성북구1', '성북구2', '성북구4', '성북구5', '노원구1', '동대문구1']),
               '성북구4' : set(['성북구1', '성북구2', '성북구3', '성북구5', '동대문구5']),
               '성북구5' : set(['성북구1', '성북구2', '성북구3', '성북구4', '종로구2']),
               '송파구1' : set(['송파구2', '송파구3', '송파구4', '송파구5', '송파구6', '광진구1']),
               '송파구2' : set(['송파구1', '송파구3', '송파구4', '송파구5', '송파구6', '강동구4']),
               '송파구3' : set(['송파구1', '송파구2', '송파구4', '송파구5', '송파구6', '강동구3']),
               '송파구4' : set(['송파구1', '송파구2', '송파구3', '송파구5', '송파구6', '강남구7']),
               '송파구5' : set(['송파구1', '송파구2', '송파구3', '송파구4', '송파구6', '강남구6']),
               '송파구6' : set(['송파구1', '송파구2', '송파구3', '송파구4', '송파구5', '강남구5']),
               '양천구1' : set(['양천구2', '양천구3', '영등포구6']),
               '양천구2' : set(['양천구1', '양천구3', '구로구2']),
               '양천구3' : set(['양천구1', '양천구2', '강서구4']),
               '영등포구1' : set(['영등포구2', '영등포구3', '영등포구4', '영등포구5', '영등포구6', '마포구1', '용산구1']),
               '영등포구2' : set(['영등포구1', '영등포구3', '영등포구4', '영등포구5', '영등포구6', '여의도']),
               '영등포구3' : set(['영등포구1', '영등포구2', '영등포구4', '영등포구5', '영등포구6', '동작구4']),
               '영등포구4' : set(['영등포구1', '영등포구2', '영등포구3', '영등포구5', '영등포구6', '금천구3']),
               '영등포구5' : set(['영등포구1', '영등포구2', '영등포구3', '영등포구4', '영등포구6', '구로구3']),
               '영등포구6' : set(['영등포구1', '영등포구2', '영등포구3', '영등포구4', '영등포구5', '양천구1']),
               '용산구1' : set(['용산구2', '용산구3', '용산구4', '용산구5', '용산구6', '영등포구1', '동작구1', '강남구1', '서초구1']),
               '용산구2' : set(['용산구1', '용산구3', '용산구4', '용산구5', '용산구6', '여의도']),
               '용산구3' : set(['용산구1', '용산구2', '용산구4', '용산구5', '용산구6', '마포구5']),
               '용산구4' : set(['용산구1', '용산구2', '용산구3', '용산구5', '용산구6', '중구3']),
               '용산구5' : set(['용산구1', '용산구2', '용산구3', '용산구4', '용산구6', '성동구2']),
               '용산구6' : set(['용산구1', '용산구2', '용산구3', '용산구4', '용산구5']),
               '은평구1' : set(['은평구2', '은평구3', '은평구4', '은평구5', '서울시 외곽']),
               '은평구2' : set(['은평구1', '은평구3', '은평구4', '은평구5', '서울시 외곽']),
               '은평구3' : set(['은평구1', '은평구2', '은평구4', '은평구5', '종로구6']),
               '은평구4' : set(['은평구1', '은평구2', '은평구3', '은평구5', '서대문구4']),
               '은평구5' : set(['은평구1', '은평구2', '은평구3', '은평구4', '마포구3']),
               '종로구1' : set(['종로구2', '종로구3', '종로구4', '종로구5', '종로구6', '서울시 외곽']),
               '종로구2' : set(['종로구1', '종로구3', '종로구4', '종로구5', '종로구6', '성북구5']),
               '종로구3' : set(['종로구1', '종로구2', '종로구4', '종로구5', '종로구6', '동대문구4']),
               '종로구4' : set(['종로구1', '종로구2', '종로구3', '종로구5', '종로구6', '중구1']),
               '종로구5' : set(['종로구1', '종로구2', '종로구3', '종로구4', '종로구6', '서대문구1']),
               '종로구6' : set(['종로구1', '종로구2', '종로구3', '종로구4', '종로구5', '은평구3']),
               '중구1' : set(['중구2', '중구3', '중구4', '중구5', '종로구4']),
               '중구2' : set(['중구1', '중구3', '중구4', '중구5', '동대문구4', '성동구3']),
               '중구3' : set(['중구1', '중구2', '중구4', '중구5', '용산구4']),
               '중구4' : set(['중구1', '중구2', '중구3', '중구5', '마포구5', '서대문구2']),
               '중구5' : set(['중구1', '중구2', '중구3', '중구4']),
               '중랑구1' : set(['중랑구2', '중랑구3', '서울시 외곽']),
               '중랑구2' : set(['중랑구1', '중랑구3', '광진구3', '동대문구2']),
               '중랑구3' : set(['중랑구1', '중랑구2', '노원구1']),
               # 강남구, 송파구, 서초구, 동작구, 영등포구, 강서구, 마포구, 용산구, 성동구, 광진구 : 10개
#                '한강' : set(['강남구1', '송파구1', '서초구1', '동작구1', '영등포구1',
#                            '강서구1', '마포구1', '용산구1', '성동구1', '광진구1']),
               # 한강을 통해 이동
               # 강서구-마포구, 영등포구-마포구, 영등포구-용산구, 동작구-용산구, 강남구-용산구, 서초구-용산구
               # 성동구-강남구, 광진구-송파구
               # 서울시 외각
               # 강서구2, 구로구1, 금천구1, 관악구1, 서초구1, 강남구1, 강동구2, 중랑구1, 강북구1, 종로구1, 은평구2 : 14개
               # 한강이랑 겹침 : 강서구, 서초구, 강남구
#                '서울시 외곽' : set(['강서구2', '강서구3', '구로구1', '금천구1', '관악구1',
#                                '서초구2', '강남구2', '강동구1', '강동구2', '중랑구1',
#                                '강북구1', '종로구1', '은평구1', '은평구2']),
               '여의도' : set(['마포구2', '영등포구2', '용산구2'])
              }

In [3]:
graph_seoul = {'강남구1' : set(['강남구2', '강남구3', '강남구4', '강남구5', '강남구6', '강남구7', '강남구8', '용산구1', '성동구1']),
               '강남구2' : set(['강남구1', '강남구3', '강남구4', '강남구5', '강남구6', '강남구7', '강남구8']),
               '강남구3' : set(['강남구1', '강남구2', '강남구4', '강남구5', '강남구6', '강남구7', '강남구8', '서초구6']),
               '강남구4' : set(['강남구1', '강남구2', '강남구3', '강남구5', '강남구6', '강남구7', '강남구8', '서초구5']),
               '강남구5' : set(['강남구1', '강남구2', '강남구3', '강남구4', '강남구6', '강남구7', '강남구8', '송파구6']),
               '강남구6' : set(['강남구1', '강남구2', '강남구3', '강남구4', '강남구5', '강남구7', '강남구8', '송파구5']),
               '강남구7' : set(['강남구1', '강남구2', '강남구3', '강남구4', '강남구5', '강남구6', '강남구8', '송파구4']),
               '강남구8' : set(['강남구1', '강남구2', '강남구3', '강남구4', '강남구5', '강남구6', '강남구7']),
               '강동구1' : set(['강동구2', '강동구3', '강동구4']),
               '강동구2' : set(['강동구1', '강동구3', '강동구4']),
               '강동구3' : set(['강동구1', '강동구2', '강동구4', '송파구3']),
               '강동구4' : set(['강동구1', '강동구2', '강동구3', '송파구2']),
               '강북구1' : set(['강북구2', '강북구3', '강북구4']),
               '강북구2' : set(['강북구1', '강북구3', '강북구4', '도봉구2']),
               '강북구3' : set(['강북구1', '강북구2', '강북구4', '성북구2']),
               '강북구4' : set(['강북구1', '강북구2', '강북구3', '성북구1']),
               '강서구1' : set(['강서구2', '강서구3', '강서구4', '강서구5', '마포구1']),
               '강서구2' : set(['강서구1', '강서구3', '강서구4', '강서구5']),
               '강서구3' : set(['강서구1', '강서구2', '강서구4', '강서구5']),
               '강서구4' : set(['강서구1', '강서구2', '강서구3', '강서구5', '양천구3']),
               '강서구5' : set(['강서구1', '강서구2', '강서구3', '강서구4']),
               '관악구1' : set(['관악구2', '관악구3', '관악구4', '관악구5']),
               '관악구2' : set(['관악구1', '관악구3', '관악구4', '관악구5', '금천구3']),
               '관악구3' : set(['관악구1', '관악구2', '관악구4', '관악구5', '구로구4']),
               '관악구4' : set(['관악구1', '관악구2', '관악구3', '관악구5', '동작구3']),
               '관악구5' : set(['관악구1', '관악구2', '관악구3', '관악구4', '서초구3']),
               '광진구1' : set(['광진구2', '광진구3', '송파구1']),
               '광진구2' : set(['광진구1', '광진구3', '성동구5']),
               '광진구3' : set(['광진구1', '광진구2', '중랑구2', '동대문구3']),
               '구로구1' : set(['구로구2', '구로구3', '구로구4', '구로구5']),
               '구로구2' : set(['구로구1', '구로구3', '구로구4', '구로구5', '양천구2']),
               '구로구3' : set(['구로구1', '구로구2', '구로구4', '구로구5', '영등포구5']),
               '구로구4' : set(['구로구1', '구로구2', '구로구3', '구로구5', '관악구3']),
               '구로구5' : set(['구로구1', '구로구2', '구로구3', '구로구4', '금천구2']),
               '금천구1' : set(['금천구2', '금천구3']),
               '금천구2' : set(['금천구1', '금천구3', '구로구5']),
               '금천구3' : set(['금천구1', '금천구2', '영등포구4', '관악구2']),
               '노원구1' : set(['노원구2', '중랑구3', '성북구3']),
               '노원구2' : set(['노원구1', '도봉구1']),
               '도봉구1' : set(['도봉구2', '노원구2']),
               '도봉구2' : set(['도봉구1', '강북구2']),
               '동대문구1' : set(['동대문구2', '동대문구3', '동대문구4', '동대문구5', '성북구3']),
               '동대문구2' : set(['동대문구1', '동대문구3', '동대문구4', '동대문구5', '중랑구2']),
               '동대문구3' : set(['동대문구1', '동대문구2', '동대문구4', '동대문구5', '광진구3']),
               '동대문구4' : set(['동대문구1', '동대문구2', '동대문구3', '동대문구5', '성동구4', '종로구3', '중구2']),
               '동대문구5' : set(['동대문구1', '동대문구2', '동대문구3', '동대문구4', '성북구4']),
               '동작구1' : set(['동작구2', '동작구3', '동작구4', '용산구1']),
               '동작구2' : set(['동작구1', '동작구3', '동작구4', '서초구4']),
               '동작구3' : set(['동작구1', '동작구2', '동작구4', '관악구4']),
               '동작구4' : set(['동작구1', '동작구2', '동작구3', '영등포구3']),
               '마포구1' : set(['마포구2', '마포구3', '마포구4', '마포구5', '강서구1', '영등포구1']),
               '마포구2' : set(['마포구1', '마포구3', '마포구4', '마포구5', '여의도']),
               '마포구3' : set(['마포구1', '마포구2', '마포구4', '마포구5', '은평구5']),
               '마포구4' : set(['마포구1', '마포구2', '마포구3', '마포구5', '서대문구3']),
               '마포구5' : set(['마포구1', '마포구2', '마포구3', '마포구4', '중구4', '용산구3']),
               '서대문구1' : set(['서대문구2', '서대문구3', '서대문구4', '종로구5']),
               '서대문구2' : set(['서대문구1', '서대문구3', '서대문구4', '중구4']),
               '서대문구3' : set(['서대문구1', '서대문구2', '서대문구4', '마포구4']),
               '서대문구4' : set(['서대문구1', '서대문구2', '서대문구3', '은평구4']),
               '서초구1' : set(['서초구2', '서초구3', '서초구4', '서초구5', '서초구6', '용산구1']),
               '서초구2' : set(['서초구1', '서초구3', '서초구4', '서초구5', '서초구6']),
               '서초구3' : set(['서초구1', '서초구2', '서초구4', '서초구5', '서초구6', '관악구5']),
               '서초구4' : set(['서초구1', '서초구2', '서초구3', '서초구5', '서초구6', '동작구2']),
               '서초구5' : set(['서초구1', '서초구2', '서초구3', '서초구4', '서초구6', '강남구4']),
               '서초구6' : set(['서초구1', '서초구2', '서초구3', '서초구4', '서초구5', '강남구3']),
               '성동구1' : set(['성동구2', '성동구3', '성동구4', '성동구5', '강남구1']),
               '성동구2' : set(['성동구1', '성동구3', '성동구4', '성동구5', '용산구5']),
               '성동구3' : set(['성동구1', '성동구2', '성동구4', '성동구5', '중구2']),
               '성동구4' : set(['성동구1', '성동구2', '성동구3', '성동구5', '동대문구4']),
               '성동구5' : set(['성동구1', '성동구2', '성동구3', '성동구4', '광진구2']),
               '성북구1' : set(['성북구2', '성북구3', '성북구4', '성북구5', '강북구4']),
               '성북구2' : set(['성북구1', '성북구3', '성북구4', '성북구5', '강북구3']),
               '성북구3' : set(['성북구1', '성북구2', '성북구4', '성북구5', '노원구1', '동대문구1']),
               '성북구4' : set(['성북구1', '성북구2', '성북구3', '성북구5', '동대문구5']),
               '성북구5' : set(['성북구1', '성북구2', '성북구3', '성북구4', '종로구2']),
               '송파구1' : set(['송파구2', '송파구3', '송파구4', '송파구5', '송파구6', '광진구1']),
               '송파구2' : set(['송파구1', '송파구3', '송파구4', '송파구5', '송파구6', '강동구4']),
               '송파구3' : set(['송파구1', '송파구2', '송파구4', '송파구5', '송파구6', '강동구3']),
               '송파구4' : set(['송파구1', '송파구2', '송파구3', '송파구5', '송파구6', '강남구7']),
               '송파구5' : set(['송파구1', '송파구2', '송파구3', '송파구4', '송파구6', '강남구6']),
               '송파구6' : set(['송파구1', '송파구2', '송파구3', '송파구4', '송파구5', '강남구5']),
               '양천구1' : set(['양천구2', '양천구3', '영등포구6']),
               '양천구2' : set(['양천구1', '양천구3', '구로구2']),
               '양천구3' : set(['양천구1', '양천구2', '강서구4']),
               '영등포구1' : set(['영등포구2', '영등포구3', '영등포구4', '영등포구5', '영등포구6', '마포구1', '용산구1']),
               '영등포구2' : set(['영등포구1', '영등포구3', '영등포구4', '영등포구5', '영등포구6', '여의도']),
               '영등포구3' : set(['영등포구1', '영등포구2', '영등포구4', '영등포구5', '영등포구6', '동작구4']),
               '영등포구4' : set(['영등포구1', '영등포구2', '영등포구3', '영등포구5', '영등포구6', '금천구3']),
               '영등포구5' : set(['영등포구1', '영등포구2', '영등포구3', '영등포구4', '영등포구6', '구로구3']),
               '영등포구6' : set(['영등포구1', '영등포구2', '영등포구3', '영등포구4', '영등포구5', '양천구1']),
               '용산구1' : set(['용산구2', '용산구3', '용산구4', '용산구5', '용산구6', '영등포구1', '동작구1', '강남구1', '서초구1']),
               '용산구2' : set(['용산구1', '용산구3', '용산구4', '용산구5', '용산구6', '여의도']),
               '용산구3' : set(['용산구1', '용산구2', '용산구4', '용산구5', '용산구6', '마포구5']),
               '용산구4' : set(['용산구1', '용산구2', '용산구3', '용산구5', '용산구6', '중구3']),
               '용산구5' : set(['용산구1', '용산구2', '용산구3', '용산구4', '용산구6', '성동구2']),
               '용산구6' : set(['용산구1', '용산구2', '용산구3', '용산구4', '용산구5']),
               '은평구1' : set(['은평구2', '은평구3', '은평구4', '은평구5']),
               '은평구2' : set(['은평구1', '은평구3', '은평구4', '은평구5']),
               '은평구3' : set(['은평구1', '은평구2', '은평구4', '은평구5', '종로구6']),
               '은평구4' : set(['은평구1', '은평구2', '은평구3', '은평구5', '서대문구4']),
               '은평구5' : set(['은평구1', '은평구2', '은평구3', '은평구4', '마포구3']),
               '종로구1' : set(['종로구2', '종로구3', '종로구4', '종로구5', '종로구6']),
               '종로구2' : set(['종로구1', '종로구3', '종로구4', '종로구5', '종로구6', '성북구5']),
               '종로구3' : set(['종로구1', '종로구2', '종로구4', '종로구5', '종로구6', '동대문구4']),
               '종로구4' : set(['종로구1', '종로구2', '종로구3', '종로구5', '종로구6', '중구1']),
               '종로구5' : set(['종로구1', '종로구2', '종로구3', '종로구4', '종로구6', '서대문구1']),
               '종로구6' : set(['종로구1', '종로구2', '종로구3', '종로구4', '종로구5', '은평구3']),
               '중구1' : set(['중구2', '중구3', '중구4', '중구5', '종로구4']),
               '중구2' : set(['중구1', '중구3', '중구4', '중구5', '동대문구4', '성동구3']),
               '중구3' : set(['중구1', '중구2', '중구4', '중구5', '용산구4']),
               '중구4' : set(['중구1', '중구2', '중구3', '중구5', '마포구5', '서대문구2']),
               '중구5' : set(['중구1', '중구2', '중구3', '중구4']),
               '중랑구1' : set(['중랑구2', '중랑구3']),
               '중랑구2' : set(['중랑구1', '중랑구3', '광진구3', '동대문구2']),
               '중랑구3' : set(['중랑구1', '중랑구2', '노원구1']),
               # 강남구, 송파구, 서초구, 동작구, 영등포구, 강서구, 마포구, 용산구, 성동구, 광진구 : 10개
#                '한강' : set(['강남구1', '송파구1', '서초구1', '동작구1', '영등포구1',
#                            '강서구1', '마포구1', '용산구1', '성동구1', '광진구1']),
               # 한강을 통해 이동
               # 강서구-마포구, 영등포구-마포구, 영등포구-용산구, 동작구-용산구, 강남구-용산구, 서초구-용산구
               # 성동구-강남구, 광진구-송파구
               # 서울시 외각
               # 강서구2, 구로구1, 금천구1, 관악구1, 서초구1, 강남구1, 강동구2, 중랑구1, 강북구1, 종로구1, 은평구2 : 14개
               # 한강이랑 겹침 : 강서구, 서초구, 강남구
#                '서울시 외곽' : set(['강서구2', '강서구3', '구로구1', '금천구1', '관악구1',
#                                '서초구2', '강남구2', '강동구1', '강동구2', '중랑구1',
#                                '강북구1', '종로구1', '은평구1', '은평구2']),
               '여의도' : set(['마포구2', '영등포구2', '용산구2'])
              }

In [4]:
# 지구 추가
def add_complex(graph, gus):
    for gu in gus:
        p = re.compile(gu + '[0-9]')
        gu_list = re.findall(p, ' '.join(list(graph.keys())))
        graph[gu] = set(gu_list)
    
        for i in gu_list:
            graph[i].add(gu)
    
    return graph

# 노드맵 최단경로 구하기
# def bfs_path(graph, start, goal):
#     queue = [(start, [start])]
#     result = []
    
#     while queue:
#         n, path = queue.pop(0)
#         if n == goal:
#             result.append(path)
#             break
#         else:
#             for m in graph[n] - set(path):
#                 queue.append((m, path + [m]))
    
#     return result[0][:]

# 최단경로를 구하는 알고리즘이 너무 느렸기 때문에 다른 알고리즘을 찾아보았음
# 최단경로 구하기 (다익스트라 알고리즘)
def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}  # start로 부터의 거리 값을 저장하기 위함
    distances[start] = 0  # 시작 값은 0이어야 함
    queue = []
    heapq.heappush(queue, [distances[start], start])  # 시작 노드부터 탐색 시작 하기 위함.

    while queue:  # queue에 남아 있는 노드가 없으면 끝
        current_distance, current_destination = heapq.heappop(queue)  # 탐색 할 노드, 거리를 가져옴.

        if distances[current_destination] < current_distance:  # 기존에 있는 거리보다 길다면, 볼 필요도 없음
            continue
    
        for new_destination in graph[current_destination]:
            distance = current_distance + 1  # 해당 노드를 거쳐 갈 때 거리
            if distance < distances[new_destination]:  # 알고 있는 거리 보다 작으면 갱신
                distances[new_destination] = distance
                heapq.heappush(queue, [distance, new_destination])  # 다음 인접 거리를 계산 하기 위해 큐에 삽입
    
    return distances

# 알고리즘 으로 구한 최단경로들을 작은순서로 정렬
def bfs_search(paths):
    new_paths = {}
    
    for i in range(len(paths)):
        for path, dist in paths[i].items():
            if path not in new_paths:
                new_paths[path] = [dist]
            else:
                new_paths[path].append(dist)

    for key, value in new_paths.items():
        value.sort()
        new_paths[key] = value

    return new_paths

# 구한 경로로 채점하기
def path_score(path, dist, n):
    secondary_node = 0
    # 인접 유효 노드 채점
    primary_node = ((-(math.atan(dist[0]/4 - 2)) + 1) / 2) * (1 / 1.054)
    
    # 간접 유효 노드 채점
    for i in range(1, len(dist)):
        secondary_node += ((math.exp(((-0.7 * (dist[i] ** 3)) / 21) + 1) / 3) + 0.2) * (1 / 1.106)
    
    # 노드 채점 : 인접 유효노드 + ((간접유효노드 / n)) - n 은 간접유효 노드의 개수
    score = round(primary_node + (secondary_node / n), 2)
    
    return [path, score]

# 위의 함수들을 한번에 처리
def node_test(graph, *jigu):
    ndt = copy.deepcopy(graph)
    jigus = list(jigu)
    paths = []
    scores = []
    
    # 지구 노드들 추가
    ndt = add_complex(ndt, jigus)
    
    # 추가한 노드들의 최단거리 구하기
    for i in range(len(jigus)):
        paths.append(dijkstra(ndt, jigus[i]))
    
    # 최단거리들을 비교하여 primary 와 secondary 를 구별하여 점수 채점
    paths_bfs = bfs_search(paths)
    for path, dist in paths_bfs.items():
        if path in jigus:
            continue
        scores.append(path_score(path, dist, len(dist)))
    
    return scores

## 노드 채점

In [5]:
# 금융지구 : 강남구, 여의도, 영등포구, 용산구
# 가장 가까운 거리가 primary, 나머지가 secondary
scores = node_test(graph_seoul, '강남구', '여의도', '영등포구', '용산구')

scores

[['강남구1', 1.35],
 ['강남구2', 1.22],
 ['강남구3', 1.22],
 ['강남구4', 1.22],
 ['강남구5', 1.22],
 ['강남구6', 1.22],
 ['강남구7', 1.22],
 ['강남구8', 1.22],
 ['강동구1', 0.92],
 ['강동구2', 0.92],
 ['강동구3', 0.98],
 ['강동구4', 0.98],
 ['강북구1', 0.49],
 ['강북구2', 0.49],
 ['강북구3', 0.61],
 ['강북구4', 0.61],
 ['강서구1', 1.06],
 ['강서구2', 0.99],
 ['강서구3', 0.99],
 ['강서구4', 0.99],
 ['강서구5', 0.99],
 ['관악구1', 0.99],
 ['관악구2', 1.04],
 ['관악구3', 0.99],
 ['관악구4', 1.01],
 ['관악구5', 1.03],
 ['광진구1', 0.99],
 ['광진구2', 1.01],
 ['광진구3', 0.92],
 ['구로구1', 1.04],
 ['구로구2', 1.04],
 ['구로구3', 1.1],
 ['구로구4', 1.04],
 ['구로구5', 1.04],
 ['금천구1', 1.04],
 ['금천구2', 1.04],
 ['금천구3', 1.1],
 ['노원구1', 0.73],
 ['노원구2', 0.61],
 ['도봉구1', 0.49],
 ['도봉구2', 0.39],
 ['동대문구1', 0.92],
 ['동대문구2', 0.92],
 ['동대문구3', 0.92],
 ['동대문구4', 1.01],
 ['동대문구5', 0.92],
 ['동작구1', 1.24],
 ['동작구2', 1.14],
 ['동작구3', 1.14],
 ['동작구4', 1.18],
 ['마포구1', 1.18],
 ['마포구2', 1.12],
 ['마포구3', 1.12],
 ['마포구4', 1.12],
 ['마포구5', 1.18],
 ['서대문구1', 0.92],
 ['서대문구2', 0.99],
 ['서대문구3', 1.01],
 ['서대문구4

In [6]:
# 채점 해서 나온 점수들의 지수를 구함
df = pd.DataFrame(scores, columns=['노드', '점수'])
df.head()

,노드,점수
0,강남구1,1.35
1,강남구2,1.22
2,강남구3,1.22
3,강남구4,1.22
4,강남구5,1.22


In [11]:
df['지수'] = round(df['점수']/df['점수'].max()*100, 2)
df.sort_values(by='지수', ascending=False)
df.head(30)

,노드,점수,지수
90,용산구1,1.42,100.00
0,강남구1,1.35,95.07
84,영등포구1,1.35,95.07
91,용산구2,1.28,90.14
92,용산구3,1.28,90.14
...,...,...,...
14,강북구3,0.61,42.96
13,강북구2,0.49,34.51
39,도봉구1,0.49,34.51
12,강북구1,0.49,34.51


In [12]:
data = pd.read_csv('data/서울특별시_부동산_면적당_가격_평균_2006-2020년.csv')
data[data['신고년도'] == 2020].sort_values(by='면적당 가격', ascending=False)

,자치구명,신고년도,면적당 가격
14,강남구,2020,1.656293e+07
224,서초구,2020,1.505349e+07
314,용산구,2020,1.325373e+07
239,성동구,2020,1.291397e+07
269,송파구,2020,1.189150e+07
359,중구,2020,1.082712e+07
194,마포구,2020,1.067973e+07
179,동작구,2020,9.968130e+06
299,영등포구,2020,9.933738e+06
29,강동구,2020,9.462772e+06
